# Установка Tacotron2

In [144]:
import os
project_path = os.getcwd().replace('\\', '/')
taco_path = project_path + '/tacotron2'
print(project_path)
print(taco_path)

C:/Users/vikto/Documents/Projects/FINAL
C:/Users/vikto/Documents/Projects/FINAL/tacotron2


In [44]:
!git clone https://github.com/NVIDIA/tacotron2.git
os.chdir(taco_path)
!git submodule init
!git submodule update
os.chdir(project_path)
!python --version

fatal: destination path 'tacotron2' already exists and is not an empty directory.


Python 3.7.13


Необходимо заменить файлы модели

# Подготовка датасета.

1. ./\*_manifest.json   ->   ./dataset.json
2. ./..some_folders../\*.flac   ->    ./mels/\*.mel

In [15]:
import json
import pandas as pd

Python 3.7.13


In [80]:
hifitts_path = 'C:/Users/vikto/Documents/Projects/final_project/data/hi_fi_tts_v0'

def read_json(json_path):
    dataset_type = json_path.split('_')[-1].replace('.json', '')
    with open(json_path, encoding='utf-8') as f:
        cond = "[" + f.read().replace("}\n{", "},\n{") + "]"
        json_data = json.loads(cond)
        for item in json_data:
            item['dataset_type'] = dataset_type
    return json_data

In [97]:
manifests = [manifest for manifest in os.listdir(hifitts_path) if 'manifest' in manifest]
manifest_paths = [f'{hifitts_path}/{manifest}' for manifest in manifests]
manifest_jsons = [read_json(manifest_path) for manifest_path in manifest_paths]
manifest_dfs = [pd.DataFrame(manifest_json) for manifest_json in manifest_jsons]
manifests_df = pd.concat(manifest_dfs, axis=0)
manifests_df

,audio_filepath,text,duration,text_no_preprocessing,text_normalized,dataset_type
0,audio/11614_other/12352/prideofjennico_01_cast...,some decision,1.03,"some decision,","some decision,",dev
1,audio/11614_other/12352/prideofjennico_01_cast...,i fear me that those around him then did not f...,7.96,I fear me that those around him then did not f...,I fear me that those around him then did not f...,dev
2,audio/11614_other/12352/prideofjennico_02_cast...,to keep myself something in countenance despit...,10.86,To keep myself something in countenance despit...,To keep myself something in countenance despit...,dev
3,audio/11614_other/12352/prideofjennico_03_cast...,under my gaze,1.06,"under my gaze,","under my gaze,",dev
4,audio/11614_other/12352/prideofjennico_04_cast...,in the vineyards,0.93,"In the vineyards,","In the vineyards,",dev
...,...,...,...,...,...,...
35141,audio/92_clean/9783/auntcretesemancipation_07_...,and john peters is at home when we get there,2.47,and John Peters is at home when we get there.,and John Peters is at home when we get there.,train
35142,audio/92_clean/9783/auntcretesemancipation_07_...,soothed in spirit,1.23,"soothed in spirit,","soothed in spirit,",train
35143,audio/92_clean/9783/auntcretesemancipation_07_...,went to bed,1.00,"went to bed,","went to bed,",train
35144,audio/92_clean/9783/auntcretesemancipation_07_...,and arose very early the next morning,2.30,"and arose very early the next morning,","and arose very early the next morning,",train


In [113]:
df = manifests_df.reset_index(drop=True).copy()
df['reader_id'] = df['audio_filepath'].apply(lambda x: x.split('/')[1].split('_')[0])
df['data_quality'] = df['audio_filepath'].apply(lambda x: x.split('/')[1].split('_')[1])
df['book_id'] = df['audio_filepath'].apply(lambda x: x.split('/')[2])
df['book_chapter'] = df['audio_filepath'].apply(lambda x: x.split('/')[3].replace('.flac', ''))
df['mel_path'] = 'mels/' + df.index.astype('string') + '_' + df['dataset_type'] + '_' + df['reader_id']
readers_list = [reader_id for reader_id in df.reader_id.unique()]
readers_dict = {reader_id: str(readers_list.index(reader_id)) for reader_id in readers_list}
df['reader_id_norm'] = df['reader_id'].apply(lambda x: readers_dict[x])
df['txt_line'] = df['mel_path'] + '|' + df['text'] + '|' + df['reader_id_norm'] + '\n'

In [121]:
df

,audio_filepath,text,duration,text_no_preprocessing,text_normalized,dataset_type,reader_id,data_quality,book_id,book_chapter,mel_path,reader_id_norm,txt_line
0,audio/11614_other/12352/prideofjennico_01_cast...,some decision,1.03,"some decision,","some decision,",dev,11614,other,12352,prideofjennico_01_castle_0028,mels/0_dev_11614,0,mels/0_dev_11614|some decision|0\n
1,audio/11614_other/12352/prideofjennico_01_cast...,i fear me that those around him then did not f...,7.96,I fear me that those around him then did not f...,I fear me that those around him then did not f...,dev,11614,other,12352,prideofjennico_01_castle_0119,mels/1_dev_11614,0,mels/1_dev_11614|i fear me that those around h...
2,audio/11614_other/12352/prideofjennico_02_cast...,to keep myself something in countenance despit...,10.86,To keep myself something in countenance despit...,To keep myself something in countenance despit...,dev,11614,other,12352,prideofjennico_02_castle_0407,mels/2_dev_11614,0,mels/2_dev_11614|to keep myself something in c...
3,audio/11614_other/12352/prideofjennico_03_cast...,under my gaze,1.06,"under my gaze,","under my gaze,",dev,11614,other,12352,prideofjennico_03_castle_0044,mels/3_dev_11614,0,mels/3_dev_11614|under my gaze|0\n
4,audio/11614_other/12352/prideofjennico_04_cast...,in the vineyards,0.93,"In the vineyards,","In the vineyards,",dev,11614,other,12352,prideofjennico_04_castle_0087,mels/4_dev_11614,0,mels/4_dev_11614|in the vineyards|0\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
323973,audio/92_clean/9783/auntcretesemancipation_07_...,and john peters is at home when we get there,2.47,and John Peters is at home when we get there.,and John Peters is at home when we get there.,train,92,clean,9783,auntcretesemancipation_07_hill_0346,mels/323973_train_92,9,mels/323973_train_92|and john peters is at hom...
323974,audio/92_clean/9783/auntcretesemancipation_07_...,soothed in spirit,1.23,"soothed in spirit,","soothed in spirit,",train,92,clean,9783,auntcretesemancipation_07_hill_0348,mels/323974_train_92,9,mels/323974_train_92|soothed in spirit|9\n
323975,audio/92_clean/9783/auntcretesemancipation_07_...,went to bed,1.00,"went to bed,","went to bed,",train,92,clean,9783,auntcretesemancipation_07_hill_0349,mels/323975_train_92,9,mels/323975_train_92|went to bed|9\n
323976,audio/92_clean/9783/auntcretesemancipation_07_...,and arose very early the next morning,2.30,"and arose very early the next morning,","and arose very early the next morning,",train,92,clean,9783,auntcretesemancipation_07_hill_0350,mels/323976_train_92,9,mels/323976_train_92|and arose very early the ...


In [164]:
import sys
import numpy as np
import soundfile as sf
import torch

sys.path.append(taco_path)
from tacotron2.utils import load_flac_to_torch
from tacotron2 import layers
from tacotron2.hparams import create_hparams

In [167]:
def flac_to_mel(line_for_create_mel):
    
    audio_path = line_for_create_mel.split('&')[0]
    mel_path = line_for_create_mel.split('&')[1]
    
    _hparams = create_hparams()
    _stft = layers.TacotronSTFT(
            _hparams.filter_length, _hparams.hop_length, _hparams.win_length,
            _hparams.n_mel_channels, _hparams.sampling_rate, _hparams.mel_fmin,
            _hparams.mel_fmax)
    
    def _load_flac_to_torch(audio_path):
        data, sampling_rate = sf.read(audio_path)
        return torch.FloatTensor(data.astype(np.float32)), sampling_rate

    def _get_mel(audio_path):
        
        audio, sampling_rate = _load_flac_to_torch(audio_path)
        if sampling_rate != _stft.sampling_rate:
            raise ValueError("{} {} SR doesn't match target {} SR".format(sampling_rate, _stft.sampling_rate))
        audio_norm = audio / _hparams.max_wav_value
        audio_norm = audio_norm.unsqueeze(0)
        audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
        melspec = _stft.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0)
        return melspec
    
    if 'hifitts' not in os.listdir(taco_path):
        os.mkdir(taco_path + '/hifitts')
        os.mkdir(taco_path + '/hifitts/mels')
    
    load_audio_path = hifitts_path + '/' + audio_path
    save_mel_path = taco_path + '/hifitts/' + mel_path
    
    melspec = _get_mel(load_audio_path)
    np.save(save_mel_path, melspec)

In [168]:
df['line_for_create_mel'] = df['audio_filepath'] + '&' + df['mel_path']
df['line_for_create_mel'].apply(lambda x: flac_to_mel(x))

KeyboardInterrupt: 